# Les optimisation en ordonnancement.

# Introduction.

Dans ce notebook, je vais principalement traiter des problèmes d'ordonnancement en usine et en atelier, cependant, on peut transposer les modèles mathématiques à d'autres cas d'utilisation.

Les principaux modèles sont ( En Anglais ) :

- Machine unique ( Single machine sheduling )
- Machines parallèles 
- Ateliers à cheminement unique (Flow Shop)
- Ateliers à cheminements multiples (Job Shop)

Je vais résoudre avec divers solveurs Python.


Etude globale proposée par Estelle Derrien - Github15000

CREATION EN COURS


# Sommaire

- 2. Machine unique
        - Notre problème de base
        - Qu'est ce que le "timespan"
        - Modélisation mathématique
        - Résolution avec Python Pulp
- 2. Machines parrallèles
        - Notre problème de base
        - Modélisation mathématique
        - Résolution
- 3. Ateliers à cheminement unique (Flow Shop)
        - Notre problème de base
        - Modélisation mathématique
        - Résolution avec Python Pulp
- 4. Ateliers à cheminements multiples (Job Shop)
        - Notre problème de base
        - Modélisation mathématique
        - Exemple
